# comercio_ext_indices.tb_nomenclatura_mercosul
> ### Origem — `bronze/autoloader/landingbeca2026jan/balancacomercial/NCM_delta`
## 📌 Descrição do arquivo

Tabela de referência da **NCM (Nomenclatura Comum do Mercosul)** com:

- Identificação do produto (NCM + descrição)
- Unidade estatística do produto
- Hierarquia SH6
- Classificações econômicas e setoriais (PPE, PPI, CUCI, ISIC, CGCE, SIIT…)
- Descrições multilíngues (PT/ES/EN)

Tipo: Dimensão Produto (hub) + bridges embutidos  
Uso: Enriquecimento e integração setorial (NCM → ISIC → CNAE)

|Coluna|Descrição técnica|Interpretação humana|
|---|---|---|
|`CO_NCM`|Código NCM do produto|Identificador do produto no COMEX|
|`CO_UNID`|Unidade estatística|Unidade oficial usada em `QT_ESTAT`|
|`CO_SH6`|Código SH 6 dígitos|Agrupamento internacional do produto (nível 6)|
|`CO_PPE`|Código PPE|Estágio da cadeia produtiva (primário/intermediário/final etc., conforme tabela PPE)|
|`CO_PPI`|Código PPI|Classificação complementar de produto (conforme tabela PPI)|
|`CO_FAT_AGREG`|Fator/agregação|Indicador técnico de agregação/agrupamento do produto|
|`CO_CUCI_ITEM`|Código CUCI item|Mapeamento do produto para classificação CUCI|
|`CO_CGCE_N3`|Código CGCE nível 3|Classificação econômica (CGCE) no nível N3|
|`CO_SIIT`|Código SIIT|Classificação complementar (SIIT)|
|`CO_ISIC_CLASSE`|Código ISIC classe|Atividade econômica associada ao produto (nível classe)|
|`CO_EXP_SUBSET`|Subconjunto exportação|Indicador de subset/segmento de exportação (uso analítico)|
|`NO_NCM_POR`|Descrição em português|Nome do produto em PT-BR|
|`NO_NCM_ESP`|Descrição em espanhol|Nome do produto em ES|
|`NO_NCM_ING`|Descrição em inglês|Nome do produto em EN|

## Configurações
> #### **imports**
> #### **get files**
> #### **schema**

In [0]:
from pyspark.sql import functions as F
from pyspark.sql import types as T
from delta.tables import DeltaTable
jvm = spark._jvm
FileSystem = jvm.org.apache.hadoop.fs.FileSystem
Path = jvm.org.apache.hadoop.fs.Path
fs = FileSystem.get(spark._jsc.hadoopConfiguration())

In [0]:
bronzePath = "/mnt/bronze/autoloader/landingbeca2026jan/balancacomercial/ncm/"
silverPath = "/mnt/silver/landingbeca2026jan/comercio_ext_indices/tb_nomenclatura_mercosul/"
silverTable = "silver_comercio_ext_indices.tb_nomenclatura_mercosul"

In [0]:
silverSchema = T.StructType([
    T.StructField("CO_NCM",        T.StringType(),   False),
    T.StructField("CO_UNID",       T.StringType(),   True ),
    T.StructField("CO_SH6",        T.StringType(),   True ),
    T.StructField("CO_PPE",        T.StringType(),   True ),
    T.StructField("CO_PPI",        T.StringType(),   True ),
    T.StructField("CO_CUCI_ITEM",  T.StringType(),   True ),
    T.StructField("CO_CGCE_N3",    T.StringType(),   True ),
    T.StructField("CO_ISIC_CLASSE",T.StringType(),   True ),
    T.StructField("TS_REF",        T.TimestampType(),False),
    T.StructField("NM_ORIGEM",     T.StringType(),   False),
])

## Extração
> #### **saprk.read**

In [0]:
df_bronze = spark.read.format("delta").load(bronzePath)

In [0]:
def up(s): return F.upper(F.trim(F.col(s).cast(T.StringType())))

df_norm = (
    df_bronze
    .withColumn("CO_NCM", up("CO_NCM"))
    .withColumn("CO_UNID", up("CO_UNID"))
    .withColumn("CO_SH6", up("CO_SH6"))
    .withColumn("CO_PPE", up("CO_PPE"))
    .withColumn("CO_PPI", up("CO_PPI"))
    .withColumn("CO_CUCI_ITEM", up("CO_CUCI_ITEM"))
    .withColumn("CO_CGCE_N3", up("CO_CGCE_N3"))
    .withColumn("CO_ISIC_CLASSE", up("CO_ISIC_CLASSE"))
    .withColumn("NO_NCM_POR", up("NO_NCM_POR"))
    .withColumn("NO_NCM_ESP", up("NO_NCM_ESP"))
    .withColumn("NO_NCM_ING", up("NO_NCM_ING"))
    .withColumn("TS_REF", F.current_timestamp())
    .withColumn("NM_ORIGEM", F.lit("/landingbeca2026jan/comercio_ext_indices/NCM_delta"))
)

##Validações
> #### **data quality**
> #### **deduplicação**
> #### **schema fit**

In [0]:
df_valid = df_norm.filter(F.col("CO_NCM").isNotNull())

In [0]:
df_dedup = df_valid.dropDuplicates(["CO_NCM"])

In [0]:
df_silver = df_dedup.select(
    "CO_NCM","NO_NCM_POR", "NO_NCM_ESP","NO_NCM_ING","CO_UNID","CO_SH6","CO_PPE","CO_PPI",
    "CO_CUCI_ITEM","CO_CGCE_N3","CO_ISIC_CLASSE","TS_REF","NM_ORIGEM"
)

##Carga
> #### **merge**

In [0]:
delta_target = DeltaTable.forName(spark, "silver_comercio_ext_indices.tb_nomenclatura_mercosul")

merge_condition = """
  t.CO_NCM = s.CO_NCM   AND
  t.CO_UNID = s.CO_UNID   AND
  t.CO_SH6 = s.CO_SH6   AND
  t.CO_PPE = s.CO_PPE  AND
  t.CO_PPI = s.CO_PPI  AND
  t.CO_CUCI_ITEM = s.CO_CUCI_ITEM AND
  t.CO_CGCE_N3 = s.CO_CGCE_N3   AND
  t.CO_ISIC_CLASSE = s.CO_ISIC_CLASSE
"""

(delta_target.alias("t")
    .merge(df_silver.alias("s"), merge_condition)
    .whenMatchedUpdate(set={
        "t.NO_NCM_POR": "s.NO_NCM_POR",
        "t.NO_NCM_ESP": "s.NO_NCM_ESP",
        "t.NO_NCM_ING": "s.NO_NCM_ING",
        "t.TS_REF": "s.TS_REF",
        "t.NM_ORIGEM": "s.NM_ORIGEM"
    })
    .whenNotMatchedInsertAll()
    .execute()
)

In [0]:
display(spark.sql("SELECT * FROM silver_comercio_ext_indices.tb_nomenclatura_mercosul"))